In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 115.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 30.6 MB/s eta 0:00:

In [4]:
!pip list | grep -E 'transformers|torch|Keras'

simpletransformers               0.64.3
torch                            2.0.1+cu118
torchaudio                       2.0.2+cu118
torchdata                        0.6.1
torchsummary                     1.5.1
torchtext                        0.15.2
torchvision                      0.15.2+cu118
transformers                     4.34.0


In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

'2.0.1+cu118'

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [8]:
import transformers
from transformers import BertForTokenClassification, AdamW

transformers.__version__

'4.34.0'

In [9]:
from transformers import get_linear_schedule_with_warmup

In [10]:
import torch
#torch.save(model, '/content/drive/MyDrive/Model/model1')

saved_model = torch.load('/content/drive/MyDrive/589/models/bert')

In [11]:
print(saved_model)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [27]:
test_sentence = """
Witness Report

Date:September 15, 2023

Time: 3:30 PM

Location: Elm Street, Cityville

Witness Information:
- Name: John Smith
- Address: 123 Main Street, Cityville
- Phone Number: (555) 123-4567
- Email: john.smith@email.com

  Incident Details:
-   Date and Time of Incident:   September 15, 2023, at approximately 3:00 PM
- Location of Incident: Elm Street, near the intersection with Oak Avenue, Cityville
- Description of Incident: I witnessed a car accident involving two vehicles at the above-mentioned location. I was driving behind both vehicles when the accident occurred.
Description of Vehicles Involved:
1. Vehicle 1
   - Make and Model: Toyota Camry
   - Color: Silver
   - License Plate: ABC 123
   - Driver: Unknown
   - Passengers: None visible

2.   Vehicle 2
   -   Make and Model:   Ford Explorer
   -   Color:   Red
   -   License Plate:   XYZ 789
   -   Driver:   Jane Doe
   -   Passengers:   Two individuals, one adult, and one child (approximately 10 years old)

  Sequence of Events:
1. I was driving behind both vehicles along Elm Street.
2. Vehicle 1 suddenly stopped at the intersection with Oak Avenue, and I observed brake lights.
3. Vehicle 2, driven by Jane Doe, was following closely behind Vehicle 1.
4. Vehicle 2 was unable to stop in time and rear-ended Vehicle 1.
5. The impact caused Vehicle 1 to move forward and hit a nearby utility pole.

  Injuries and Assistance:
- I immediately pulled over and approached the scene to check on the occupants of both vehicles.
- Jane Doe and the child in Vehicle 2 appeared shaken but did not seem to have any visible injuries.
- The driver of Vehicle 1 was not present at the scene.

  Actions Taken:
- I called 911 to report the accident and request medical assistance.
- I provided my contact information to Jane Doe and remained at the scene until the police and medical personnel arrived.
- I shared my observations and the sequence of events with the responding police officer.

  Additional Information:
- Weather conditions were clear, and the road appeared to be dry at the time of the accident.
- There were no unusual circumstances or hazards in the vicinity that could have contributed to the accident.

"""
test_sentence = "Artificial Intelligence is the next way forward/"


In [28]:
tokenized_sentence = tokenizer.encode(test_sentence)
print(tokenized_sentence)
input_ids = torch.tensor([tokenized_sentence]).cuda()

[101, 2051, 19814, 2916, 7829, 1110, 1103, 1397, 1236, 1977, 120, 102]


In [29]:
with torch.no_grad():
    output = saved_model(input_ids)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)


In [30]:
tag2idx = {'B':0,
 'I':1,
 'O':2,
  'X':3,
 '[CLS]':4,
 '[SEP]':5,
 'PAD':6}

In [31]:
tag_values = list(tag2idx.keys())

In [32]:
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
print(tokens)
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(tag_values[label_idx])
        new_tokens.append(token)


['[CLS]', 'Art', '##ific', '##ial', 'Intelligence', 'is', 'the', 'next', 'way', 'forward', '/', '[SEP]']


In [33]:
for token, label in zip(new_tokens, new_labels):
    print("{}\t\t\t{}".format(label, token))

O			[CLS]
O			Artificial
O			Intelligence
O			is
O			the
O			next
O			way
O			forward
O			/
O			[SEP]
